In [34]:
import cv2
import numpy as np

import math
import serial
import time

from collections import deque
from sklearn.linear_model import LinearRegression

# YOLOv3 관련 파일 경로 설정
weights_path = 'yolov3/yolov3.weights'
config_path = 'yolov3/yolov3.cfg'
labels_path = 'yolov3/coco.names'

# YOLOv3 모델 초기화
net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# 객체 탐지에 사용할 클래스 레이블 로딩
with open(labels_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# 영상 스트림 가져오기
stream_url = 'http://192.168.0.114:81/stream'  # 스트림 URL 설정
cap = cv2.VideoCapture(stream_url)

frame_count = 0  # 현재 프레임 수 초기화

# 영상의 가로, 세로 크기 확인
_, frame = cap.read()
height, width, _ = frame.shape

# 윈도우 크기 2배로 변경
window_width = width 
window_height = height 

# 각도 변수 초기화
phi, theta = 90, 0

# 최신 3개 값을 유지하기 위한 덱 초기화
max_len=3
q_phi = deque([0] * max_len, maxlen=max_len)
q_theta = deque([0] * max_len, maxlen=max_len)

# 시리얼 통신 객체 초기화
py_serial = serial.Serial(
    port='COM4',
    baudrate=9600,
)

# Pan-tilt 위상 변이기
def moudle_phaseShiftSupport():
    global q_phi, q_theta, max_len
    
    # 주어진 데이터
    X = np.arange(1, max_len + 1).reshape(-1, 1)
    y_phi = np.array(q_phi)
    y_theta = np.array(q_theta)
    
    # 선형 회귀 모델 훈련
    model = LinearRegression()

    # x =(2틱 뒤)에 대한 예측
    x_pred = np.array([[max_len+2]])
    
    # phi 예측
    model.fit(X, y_phi)
    y_pred_phi = model.predict(x_pred)
    
    # theta 예측
    model.fit(X, y_theta)
    y_pred_theta = model.predict(x_pred)

    return y_pred_phi,y_pred_theta

# 중앙 십자 표시 함수
def draw_crosshair(frame, color):
    center_x = window_width // 2
    center_y = window_height // 2
    cv2.line(frame, (center_x - 10, center_y), (center_x + 10, center_y), color, 2)
    cv2.line(frame, (center_x, center_y - 10), (center_x, center_y + 10), color, 2)

# 클릭한 지점을 중앙에 오도록 이동하는 함수
def move_to_center(x, y):
    global phi, theta
    
    center_x = window_width // 2
    center_y = window_height // 2

    delta_x = center_x - x
    delta_y = center_y - y

    phi += math.degrees(math.asin(delta_x / center_x)) * (68/180) * (32/24)
    if phi < 0:
        phi = 0
    elif phi > 180:
        phi = 180
    theta += math.degrees(math.asin(delta_y / center_y)) * (68/180)
    if theta < 0:
        theta = 0
    elif theta > 180:
        theta = 180
    
    # 덱에 저장
    q_phi.append(phi)
    q_theta.append(theta)
    
    command = f'{phi},{theta}\n'
    py_serial.write(command.encode())
    time.sleep(0.1)

    # 윈도우에 중앙 십자 표시
    draw_crosshair(frame, (0, 255, 0))

# 각도 변경만 전달
def command_to_rotate(phi, theta):
    if phi < 0:
        phi = 0
    elif phi > 180:
        phi = 180
    if theta < 0:
        theta = 0
    elif theta > 180:
        theta = 180
        
    command = f'{phi},{theta}\n'
    py_serial.write(command.encode())
    time.sleep(0.1)
    
# 드론 탐지 여부를 저장하는 변수
drone_detected = False
drone_lost_frames = 0

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1

    if frame_count % 15 != 0:  # 0.25 초 1프레임 ( 초당 60 프레임 )
        continue

    # 영상 프레임 크기 조정
    frame = cv2.resize(frame, (window_width, window_height))  # 크기를 2배로 확대

    # YOLOv3 입력 이미지 전처리
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # 객체 탐지 결과 처리
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.7:  # 탐지 임계값 설정 (0.7 이상일 때만 탐지)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 객체의 경계 상자 좌표 계산
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                
                center_x = x + w / 2
                center_y = y + h / 2
                
                move_to_center(center_x,center_y)
                
                drone_x = w
                drone_y = h
                
                S = (5 * math.cos(math.radians(phi)) + 15 * math.sin(math.radians(phi))) / 15
                R = 38.5 / h
                
                confidences.append(float(confidence))
                class_ids.append(class_id)
                
                drone_detected = True

    if not drone_detected:
        drone_lost_frames += 1

    if drone_detected and drone_lost_frames >= 2 * 15:  # 2초 동안 드론이 탐지되지 않은 경우
        drone_detected = False
        pred_phi, pred_theta = moudle_phaseShiftSupport()
        command_to_rotate(*pred_phi, *pred_theta)

    if drone_detected:
        drone_lost_frames = 0

    # 비최대 억제 수행 (같은 객체에 대한 중복 탐지 제거)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # 탐지된 객체를 화면에 표시
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = f'{classes[class_ids[i]]}: {confidences[i]:.2f}'
            label_S = f'R: {S:.2f}'
            label_P = f'phi: {phi:.2f}'
            label_T = f'theta: {theta:.2f}'
            color = (0, 0, 255)  # 객체 경계 상자의 색상 설정
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label, (x, y - 10), font, 0.5, color, 2)
            cv2.putText(frame, label_S, (x, y + h + 20), font, 0.5, color, 2)
            cv2.putText(frame, label_P, (x, y + h + 40), font, 0.5, color, 2)
            cv2.putText(frame, label_T, (x, y + h + 60), font, 0.5, color, 2)

    # 화면에 영상 프레임 출력
    cv2.imshow('Object Detection', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# 시리얼 통신 객체 해제
py_serial.close()

# 리소스 해제
cap.release()
cv2.destroyAllWindows()